כדי להריץ את המחברות הבאות, אם עדיין לא עשית זאת, עליך לפרוס מודל שמשתמש ב־`text-embedding-ada-002` כמודל בסיס ולהגדיר את שם הפריסה בתוך קובץ ‎.env כ־`AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

לאחר מכן, נטען את אינדקס ההטמעות אל תוך Dataframe של Pandas. אינדקס ההטמעות מאוחסן בקובץ JSON בשם `embedding_index_3m.json`. אינדקס ההטמעות מכיל את ההטמעות עבור כל תמלולי YouTube עד לסוף אוקטובר 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

בהמשך, ניצור פונקציה בשם `get_videos` שתחפש את השאילתה בתוך אינדקס ההטמעות. הפונקציה תחזיר את חמשת הסרטונים שהכי דומים לשאילתה. כך הפונקציה פועלת:

1. קודם כל, נוצרת עותק של אינדקס ההטמעות.
2. לאחר מכן, מחושבת ההטמעה עבור השאילתה בעזרת OpenAI Embedding API.
3. אז נוצרת עמודה חדשה באינדקס ההטמעות בשם `similarity`. עמודת `similarity` מכילה את הדמיון הקוסיני בין ההטמעה של השאילתה לבין ההטמעות של כל קטע וידאו.
4. בשלב הבא, אינדקס ההטמעות מסונן לפי עמודת `similarity`. האינדקס מסונן כך שיכלול רק סרטונים עם דמיון קוסיני של 0.75 ומעלה.
5. לבסוף, אינדקס ההטמעות ממויין לפי עמודת `similarity` ומוחזרים חמשת הסרטונים המובילים.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

הפונקציה הזו מאוד פשוטה, היא פשוט מדפיסה את תוצאות שאילתת החיפוש.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ראשית, אינדקס ההטמעות נטען לתוך Dataframe של Pandas.
2. לאחר מכן, המשתמש מתבקש להזין שאילתה.
3. אז פונקציית `get_videos` נקראת כדי לחפש את השאילתה באינדקס ההטמעות.
4. לבסוף, פונקציית `display_results` נקראת כדי להציג את התוצאות למשתמש.
5. המשתמש מתבקש להזין שאילתה נוספת. התהליך הזה ממשיך עד שהמשתמש מזין `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.he.png)

תתבקשו להזין שאילתה. הזינו שאילתה ולחצו על Enter. האפליקציה תחזיר רשימה של סרטונים שרלוונטיים לשאילתה. בנוסף, האפליקציה תספק קישור למקום בסרטון שבו נמצאת התשובה לשאלה.

הנה כמה שאילתות שכדאי לנסות:

- מה זה Azure Machine Learning?
- איך רשתות עצביות קונבולוציוניות עובדות?
- מהי רשת עצבית?
- האם אפשר להשתמש ב-Jupyter Notebooks עם Azure Machine Learning?
- מה זה ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

Certainly! Here is your text translated into Hebrew:

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור ייחשב כמקור הסמכותי. למידע קריטי, מומלץ לפנות לתרגום מקצועי על ידי אדם. איננו אחראים לכל אי-הבנה או פירוש שגוי הנובעים מהשימוש בתרגום זה.
